El método de cuadratura consiste en sustituir la partición regular que se utiliza en métodos como el del trapecio o el de Simpson, por una que sea mas eficiente. Resulta, que si se utilizan las raices del n-esimo polinomio de Legendre para hacer la interpolación de una integral en el intervalo $[-1,1],$ la aproximación de la integral será exacta para polinomios de grado hasta $2n$, por supuesto, no todas las integrales se realizan en ese intervalo, pero con un cambio de variable apropiado se podrían cambiar los límites a -1 y 1. Veamos como tendría que ser dicho cambio:

Buscamos una función lineal $T:\left[a,b\right]\to \left[-1,1\right]$

La función es de forma $$T(x) = mx+k$$

\begin{aligned}
& T(a) = -1 \to ma+k = -1\\
& T(b) = 1 \to mb +k = 1
\end{aligned}

Resolviendo el sistema nos queda

$$T(x) = \frac{2}{b-a}x -\frac{b+a}{b-a}$$

cuando hacemos esa sustitución la integral de $a$ a $b$ quedaria

$$\int_{a}^b f(x)dx = \frac{b-a}{2}\int_{-1}^1 f\left(\frac{b-a}{2}\left( v+\frac{b+a}{b-a}\right)\right)dv$$








In [2]:
function xSubscriptSym(i)
   a = "x_"
   b = string(i)
    
    return string(a,b)
    
end

xSubscriptSym (generic function with 1 method)

In [18]:
using SymPy
#=Copiamos nuestra función para la interpolación de la grange N va a ser en este caso el numero de
puntos=#
function lagrangeInterpolationDefined(N,f)
    x0 = Sym("x_0")
    x = Sym("x")
    g = f(x)
    points = []
    push!(points,x0)
    #=We create all the symbols for each point=#
    
    for i in 1:N-1
       push!(points,Sym(xSubscriptSym(i))) 
    end
    
    sum = SymFunction("sum")
    lagrangeTerm = SymFunction("lagrangeTerm")
    lagrangeTerm = 1
    sum = 0
    
    for i in 1:N
        lagrangeTerm  = 1
        for j in 1:N
            if(j!=i)
                
               lagrangeTerm = lagrangeTerm*(x-points[j])/((points[i])-points[j])
            
            end
    end
          sum+=g(points[i])*lagrangeTerm
end
    return sum 
end

lagrangeInterpolationDefined (generic function with 1 method)

In [24]:
#=Ahora construimos la aproximación de la integral =#
using SymPy.SpecialFuncs
function quadratureMethod(f,a,b,n)
   #=Sacamos el n-esimo polinomio de legendre=#
    x = Sym("x")
    v = Sym("v")
    legend = legendre(n,x)
    
    #=Sacamos las raices del polinomio=#
    legendreRoots = solve(legend)
    
    #=Hacemos el cambio de variable en la función x=#
    g = f(x)
    g = subs(g,x=>((b-a)/2)*(v +((b-a)/(b+a))))
    
    #=Bien, ahora hacemos la interpolación de f =#
    lagrange = lagrangeInterpolationDefined(n,g)
    
    #=Sustituimos las raices en el polinomio=#
    
    for i in 1:n
       lagrange = subs(lagrange,Sym(xSubscriptSym(i-1))=>legendreRoots[i]) 
    end
    
    integral = integrate(lagrange,(x,-1,1))
    
    return N(integral)
end

quadratureMethod (generic function with 1 method)

In [25]:
quadratureMethod(x->x^2,0,2,2)

2.666666666666666

In [28]:
#=Si usamos tres puntos para la interpolacion, la integral sera exacta hasta para polinomios de orden 5, la siguiente integral de x^5 de 0 a 2 debe dar 64/6 = 10.6666=#

quadratureMethod(x->x^5,0,2,3)

10.666666666666668